In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install nltk 

In [8]:
# Importing Libraries
import torch
import pandas as pd
import numpy as np
import string
import re
from tqdm.auto import tqdm
tqdm.pandas()

import nltk
nltk.download('punkt')

# Checking to see if CUDA is available if so storing details in device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import psutil

workers = psutil.cpu_count()
# psutil.cpu_count(logical=False)

In [10]:
import pandas as pd

from datetime import datetime
from collections import Counter
import torch


import re  
from gensim.models import Word2Vec    #Import gensim Word2Fec
from sklearn.decomposition import PCA #Grab PCA functions
import numpy as np

#Plot helpers
import matplotlib.pyplot as plt

#Enable matplotlib to be interactive (zoom etc)
import ast

#Loading Data and Lexicon

In [18]:
df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC Emotion Intensity.csv")

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9921 entries, 0 to 9920
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   word                     9921 non-null   object 
 1   emotion                  9921 non-null   object 
 2   emotion-intensity-score  9921 non-null   float64
 3   embedding                9921 non-null   object 
dtypes: float64(1), object(3)
memory usage: 310.2+ KB


In [20]:
print(df.groupby('emotion').count())
print(df.groupby('emotion').sum())

              word  emotion-intensity-score  embedding
emotion                                               
anger         1483                     1483       1483
anticipation   864                      864        864
disgust       1094                     1094       1094
fear          1765                     1765       1765
joy           1268                     1268       1268
sadness       1298                     1298       1298
surprise       585                      585        585
trust         1564                     1564       1564
              emotion-intensity-score
emotion                              
anger                         740.397
anticipation                  431.978
disgust                       547.167
fear                          881.033
joy                           635.373
sadness                       650.952
surprise                      292.496
trust                         782.109


#WelFake

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/WELFake/WELFake.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      72134 non-null  int64 
 1   title   71576 non-null  object
 2   text    72095 non-null  object
 3   label   72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [ ]:
df.head()

,id,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
dff['total_text'] = dff['title']+ ' ' + dff['text']

In [ ]:
dff = dff[['label','total_text','id']]

#Liar

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/LIAR/Liar_all.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12791 entries, 0 to 12790
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  12791 non-null  object 
 1   label                               12791 non-null  object 
 2   statement                           12791 non-null  object 
 3   subject(s)                          12789 non-null  object 
 4   speaker                             12789 non-null  object 
 5   speaker_job_title                   9224 non-null   object 
 6   state                               10042 non-null  object 
 7   party                               12789 non-null  object 
 8   credit_history_count_barely_true    12789 non-null  float64
 9   credit_history_count_false          12789 non-null  float64
 10  credit_history_count_half_true      12789 non-null  float64
 11  credit_history_count_mostly_true    12789

In [ ]:
dff.head()

,ID,label,statement,subject(s),speaker,speaker_job_title,state,party,credit_history_count_barely_true,credit_history_count_false,credit_history_count_half_true,credit_history_count_mostly_true,credit_history_count_pants_on_fire,context,split
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30.0,30.0,42.0,23.0,18.0,Radio interview,test
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2.0,1.0,0.0,0.0,0.0,a news conference,test
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63.0,114.0,51.0,37.0,61.0,comments on ABC's This Week.,test
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1.0,1.0,3.0,1.0,1.0,a radio show,test
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5.0,7.0,2.0,2.0,7.0,a web video,test


In [ ]:
dff['statement'][0]

'Building a wall on the U.S.-Mexico border will take literally years.'

#FA-KES

In [ ]:
dff = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/FA-KES/FA-KES.csv", encoding = "ISO-8859-1")

dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   unit_id          789 non-null    int64 
 1   article_title    789 non-null    object
 2   article_content  789 non-null    object
 3   source           789 non-null    object
 4   date             789 non-null    object
 5   location         789 non-null    object
 6   label            789 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 43.3+ KB


In [ ]:
dff.tail()

,unit_id,article_title,article_content,source,date,location,label
784,1965511221,Turkish Bombardment Kills 20 Civilians in Syria,28-08-2016 Turkish Bombardment Kills 20 Civili...,manar,8/28/2016,aleppo,0
785,1965511222,Martyrs as Terrorists Shell Aleppos Salah Eddin,17-08-2016 Martyrs as Terrorists Shell Aleppos...,manar,8/1/2016,aleppo,0
786,1965511224,Chemical Attack Kills Five Syrians in Aleppo SANA,03-08-2016 Chemical Attack Kills Five Syrians ...,manar,8/3/2016,aleppo,1
787,1965511226,5 Killed as Russian Military Chopper Shot down...,01-08-2016 5 Killed as Russian Military Choppe...,manar,8/1/2016,idlib,0
788,1965511231,Syrian Army Kills 48 ISIL Terrorists in Deir E...,April 6 2017 Syrian Army Kills 48 ISIL Terrori...,manar,4/4/2017,deir ezzor,0


In [ ]:
dff['total_text'] = dff['article_title']+ ' ' + dff['article_content']

In [ ]:
dff['total_text'][0]

#COVID19

In [ ]:
dff = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/fake_new_dataset.csv")

dff.info()

FileNotFoundError: ignored

In [ ]:
dff = dff.loc[:, ~dff.columns.str.contains('^Unnamed')]

In [ ]:
dff.head()

,title,text,subcategory,label
0,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,Facebook has shuttered a popular group for Mic...,false news,0
1,Other Viewpoints: COVID-19 is worse than the flu,We can now officially put to rest all comparis...,true,1
2,Bermuda's COVID-19 cases surpass 100,The Ministry of Health in Bermuda has confirme...,true,1
3,Purdue University says students face 'close to...,"Purdue University President Mitch Daniels, the...",partially false,0
4,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,Locking down much of the country may have help...,false news,0


In [ ]:
dff.tail()

,title,text,subcategory,label
3114,2019-nCoV: Health dept. on full alert,Quarantine advised for 80 persons across the S...,true,1
3115,Screening machine for corona virus launched at...,South Sudan has begun screening all travelers ...,true,1
3116,Coronavirus (2019-nCoV),Coronavirus (2019-nCoV) is a new respiratory i...,true,1
3117,Characterizing Patients Hospitalized With COVI...,Characterizing Patients Hospitalized With COVI...,false news,0
3118,Corona Virus WARNING,Just something to be prepared for… but then it...,false news,0


In [ ]:
dff['id'] = dff.index

In [ ]:
dff['subcategory'].unique()

array(['false news', 'true', 'partially false'], dtype=object)

In [ ]:
dff['labell'] = dff['label']

In [ ]:
dff.loc[dff['label'] == 0, 'labell'] = 1
dff.loc[dff['label'] == 1, 'labell'] = 0

In [ ]:
dff.loc[(dff['label'] == 0) & (dff['labell'] == 1)]

,title,text,subcategory,label,id,labell
0,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,Facebook has shuttered a popular group for Mic...,false news,0,0,1
3,Purdue University says students face 'close to...,"Purdue University President Mitch Daniels, the...",partially false,0,3,1
4,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,Locking down much of the country may have help...,false news,0,4,1
5,NY TIMES WORRIES HALF OF AMERICANS WILL REFUSE...,The New York Times has published an article by...,false news,0,5,1
7,CDC adds 6 new possible symptoms of Covid-19,"When the coronavirus outbreak began, the Cente...",false news,0,7,1
...,...,...,...,...,...,...
3103,Twinbrook ‘pausing’ COVID-19 plan after criticism,"Apr 24, 2020 at 4:31 PM Nursing home faced bac...",false news,0,3103,1
3109,"Experts suggest ventilation, not isolation is ...",The Coronavirus epidemic for the first time re...,false news,0,3109,1
3110,Corona virus and pentagon Quarantine help? Com...,Here is the technique we use to heal unseen ne...,false news,0,3110,1
3117,Characterizing Patients Hospitalized With COVI...,Characterizing Patients Hospitalized With COVI...,false news,0,3117,1


In [ ]:
dff['label'] = dff['labell']

In [ ]:
dff.drop('labell', axis=1, inplace=True)

In [ ]:
dff.tail()

,title,text,subcategory,label,id
3114,2019-nCoV: Health dept. on full alert,Quarantine advised for 80 persons across the S...,true,0,3114
3115,Screening machine for corona virus launched at...,South Sudan has begun screening all travelers ...,true,0,3115
3116,Coronavirus (2019-nCoV),Coronavirus (2019-nCoV) is a new respiratory i...,true,0,3116
3117,Characterizing Patients Hospitalized With COVI...,Characterizing Patients Hospitalized With COVI...,false news,1,3117
3118,Corona Virus WARNING,Just something to be prepared for… but then it...,false news,1,3118


In [ ]:
# dff.to_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/COVID19_test.csv",index=False)

In [ ]:
dff = pd.read_csv('/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/COVID19_test.csv')

dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3119 entries, 0 to 3118
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        3118 non-null   object
 1   text         3119 non-null   object
 2   subcategory  3119 non-null   object
 3   label        3119 non-null   int64 
 4   id           3119 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 122.0+ KB


In [ ]:
dff.head()

,title,text,subcategory,label,id
0,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,Facebook has shuttered a popular group for Mic...,false news,1,0
1,Other Viewpoints: COVID-19 is worse than the flu,We can now officially put to rest all comparis...,true,0,1
2,Bermuda's COVID-19 cases surpass 100,The Ministry of Health in Bermuda has confirme...,true,0,2
3,Purdue University says students face 'close to...,"Purdue University President Mitch Daniels, the...",partially false,1,3
4,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,Locking down much of the country may have help...,false news,1,4


In [ ]:
dff['total_text'] = dff['title'].astype(str) + ' ' + dff['text'].astype(str)

In [ ]:
dff['total_text'][0]

'FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP WITH 380,000 MEMBERS Facebook has shuttered a popular group for Michiganders who oppose their governor’s extreme lockdown measures, fueling debate about free speech during the coronavirus crisis.\n\n‘Michiganders Against Excessive Quarantine’ had more than 380,000 members, with media reports describing it as one of the largest anti-lockdown groups in the nation. It’s not clear if Facebook has provided an explanation for the group’s removal.\n\nVisiting the page gives the following message: “Sorry, this content isn’t available right now.”\n\nGarrett Soldano of Mattawan, Michigan created Michiganders Against Excessive Quarantine in April after becoming disenchanted with Governor Gretchen Whitmer’s stay-at-home order – a policy which he initially supported.\n\nThe group made headlines around the world after organizing ‘Operation Gridlock’, a protest in which thousands of cars created bumper-to-bumper traffic in the capital, Lansing.\n\nProtes

In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3118 entries, 0 to 3118
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        3118 non-null   object
 1   text         3118 non-null   object
 2   subcategory  3118 non-null   object
 3   label        3118 non-null   int64 
 4   id           3118 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 146.2+ KB


In [ ]:
dff.dropna(subset=['title'], how='all', inplace=True)

#ISOT

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/ISOT/ISOT.csv")

dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
 5   id       44898 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.1+ MB


In [ ]:
dff.loc[dff['title'] == ' Drunk Bragging Trump Staffer Started Russian Collusion Investigation']

,title,text,subject,date,label,id
21418,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1,21419


In [ ]:
dff.tail()

,title,text,subject,date,label,id
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1,44894
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1,44895
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1,44896
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1,44897
44897,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",1,44898


In [ ]:
dff['total_text'] = dff['title']+ ' ' + dff['text']

In [ ]:
dff['text'][30]

'(Reuters) - Democrat Doug Jones’ surprise victory over Republican Roy Moore in this month’s special U.S. Senate election will be certified on Dec. 28, Alabama state officials said on Friday. Jones will be the first Democrat sent to the Senate from Republican stronghold Alabama in a quarter century. When he takes office, Republicans’ majority in the chamber will narrow to 51 of the 100 seats. Alabama Governor Kay Ivey, Attorney General Steve Marshall and Secretary of State John Merrill will meet to certify Jones’ win, Merrill’s office said in a statement. Jones’ margin of victory was 1.5 percentage points. Moore has not conceded defeat in the Dec. 12 vote, despite being urged by President Donald Trump to do so. Calls and emails to Moore’s campaign spokeswomen were not immediately returned on Friday. Moore was a controversial candidate whose campaign was beset by allegations that he sexually assaulted or pursued teenage girls while he was in his 30s. He denied the misconduct allegations

In [ ]:
dff['total_text'][30]

'Alabama to certify Democrat Jones winner of Senate election (Reuters) - Democrat Doug Jones’ surprise victory over Republican Roy Moore in this month’s special U.S. Senate election will be certified on Dec. 28, Alabama state officials said on Friday. Jones will be the first Democrat sent to the Senate from Republican stronghold Alabama in a quarter century. When he takes office, Republicans’ majority in the chamber will narrow to 51 of the 100 seats. Alabama Governor Kay Ivey, Attorney General Steve Marshall and Secretary of State John Merrill will meet to certify Jones’ win, Merrill’s office said in a statement. Jones’ margin of victory was 1.5 percentage points. Moore has not conceded defeat in the Dec. 12 vote, despite being urged by President Donald Trump to do so. Calls and emails to Moore’s campaign spokeswomen were not immediately returned on Friday. Moore was a controversial candidate whose campaign was beset by allegations that he sexually assaulted or pursued teenage girls w

In [ ]:
dff = dff[['label','total_text','id']]

#FakeNewsNet

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/FakeNewsNet/FakeNewsNet_All.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         23196 non-null  object
 1   news_url   22866 non-null  object
 2   title      23196 non-null  object
 3   tweet_ids  21695 non-null  object
 4   label      23196 non-null  object
 5   source     22140 non-null  object
 6   id_1       23196 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


In [ ]:
dff['source'].fillna('politifact',inplace=True)

In [ ]:
dff.head()

,id,news_url,title,tweet_ids,label,source,id_1
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,Fake,gossipcop,0
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,Fake,gossipcop,1
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,Fake,gossipcop,2
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,Fake,gossipcop,3
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,Fake,gossipcop,4


In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         23196 non-null  object
 1   news_url   22866 non-null  object
 2   title      23196 non-null  object
 3   tweet_ids  21695 non-null  object
 4   label      23196 non-null  object
 5   source     23196 non-null  object
 6   id_1       23196 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


In [ ]:
dff = dff[['title','label','id_1','source','id']]

#Kaggle Real_Fake

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Kaggle_real_fake/fake_or_real_news.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
 3   id      6335 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [ ]:
dff['title'][0]

'You Can Smell Hillary’s Fear'

In [ ]:
dff.head()

,title,text,label,id
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,2
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,3
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,4


In [ ]:
dff['total_text'] = dff['title']+ ' ' + dff['text']

In [ ]:
dff = dff[['label','id','total_text']]

#Covid CodeLab

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/CodaLab Covid/Constraint_English_All.csv")

dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10700 entries, 0 to 10699
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10700 non-null  object
 1   tweet   10700 non-null  object
 2   label   10700 non-null  object
 3   split   10700 non-null  object
dtypes: object(4)
memory usage: 334.5+ KB


In [ ]:
dff.head()

,id,tweet,label,split
0,test1,Our daily update is published. States reported...,real,test
1,test2,Alfalfa is the only cure for COVID-19.,fake,test
2,test3,President Trump Asked What He Would Do If He W...,fake,test
3,test4,States reported 630 deaths. We are still seein...,real,test
4,test5,This is the sixth time a global health emergen...,real,test


#Input Embeddings - Transformer

In [ ]:
dff['text'] = dff['title']
dff.head()

,title,text,subcategory,label,id
0,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,false news,1,0
1,Other Viewpoints: COVID-19 is worse than the flu,Other Viewpoints: COVID-19 is worse than the flu,true,0,1
2,Bermuda's COVID-19 cases surpass 100,Bermuda's COVID-19 cases surpass 100,true,0,2
3,Purdue University says students face 'close to...,Purdue University says students face 'close to...,partially false,1,3
4,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,false news,1,4


In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3118 entries, 0 to 3118
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        3118 non-null   object
 1   text         3118 non-null   object
 2   subcategory  3118 non-null   object
 3   label        3118 non-null   int64 
 4   id           3118 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 146.2+ KB


In [ ]:
dff = dff.dropna(subset = ['text'], how='all')

def lower(text):
  return text.lower()

dff['text'] = dff['text'].apply(lower)

In [ ]:
import string
import re

# remove urls
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_nonascii(sent):
  return "".join([i for i in sent if i.isascii()])

def remove_punctuations(text):
  # text = text.translate(str.maketrans('', ' ', string.punctuation))
  # return text

  res = re.sub(r'[^\w\s]', ' ', text)
  return res

def remove_num(text):
  return "".join([c for c in text if not c.isdigit()])

# remove multiple spaces
def remove_mul_space(text):
  return " ".join(text.split())

def clean(text):
  text = remove_urls(text)
  text = remove_nonascii(text)
  text = remove_punctuations(text)
  text = remove_num(text)
  text = remove_mul_space(text)
  return text

In [ ]:
dff['text'] = dff['text'].apply(clean)
dff = dff.reset_index(drop=True)

In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        3118 non-null   object
 1   text         3118 non-null   object
 2   subcategory  3118 non-null   object
 3   label        3118 non-null   int64 
 4   id           3118 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 121.9+ KB


In [ ]:
dff.tail()

,title,text,subcategory,label,id
3113,2019-nCoV: Health dept. on full alert,ncov health dept on full alert,true,0,3114
3114,Screening machine for corona virus launched at...,screening machine for corona virus launched at...,true,0,3115
3115,Coronavirus (2019-nCoV),coronavirus ncov,true,0,3116
3116,Characterizing Patients Hospitalized With COVI...,characterizing patients hospitalized with covid,false news,1,3117
3117,Corona Virus WARNING,corona virus warning,false news,1,3118


In [ ]:
dff['text'][0]

'facebook deletes michigan anti lockdown group with members'

#Emotion Extraction Pipeline

In [ ]:
pandarallel.initialize()

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
def get_nearest_neighbours(index):
    tuples = []
    
    embeding = dff['embedding'][index]
    # sentence_tokens = dff['text'][index].split(' ')


    # for i,row_e in df.iterrows():
        
    #     dis = cosine_similarity([row_e['embedding']], [embeding])
    #     # print([row_e['tokens'],row_d['tokens'],dis])
    #     tuples.append([row_e['word'],row_e['emotion'],dis,row_e['embedding'],row_e['emotion-intensity-score']])

    indexes, distances = t.get_nns_by_vector(embeding, 50, include_distances=True)
    for i in range(len(indexes)):
      tuples.append([df['word'][indexes[i]],df['emotion'][indexes[i]],distances[i],df['embedding'][indexes[i]],df['emotion-intensity-score'][indexes[i]]
                    #  df['Positive'][indexes[i]],df['Negative'][indexes[i]]
                     ])
    # print(indexes)
    # print(distances)
    # print(tuples)
    
    s_tup = sorted(tuples, key=lambda x: x[2])#sort tuples based on the cosine distance
    neaarest_neighbs_words = []
    neaarest_neighbs_embs = []
    neaarest_neighbs_labels = []
    neaarest_neighbs_distance = []
    neaarest_neighbs_intensity_score = []
    for i,m in enumerate(s_tup[::-1]):
        # print(m)
        if(i<50):#getting the nearest 50 neighbours
            neaarest_neighbs_words.append(m[0])
            neaarest_neighbs_embs.append(m[3])
            neaarest_neighbs_labels.append(m[1])
            # distance = m[2].tolist()[0][0]
            distance = m[2]
            neaarest_neighbs_distance.append(distance)
            neaarest_neighbs_intensity_score.append(m[4])

    n_score_dict = calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score)
    
    neighbour_output = [n_score_dict,{'words':neaarest_neighbs_words,'embs':neaarest_neighbs_embs,'labels':neaarest_neighbs_labels}]
    normalized_score_dict = neighbour_output[0]
  
    # for key in normalized_score_dict:    
    #   normalized_score_dict[key] = round(normalized_score_dict[key],4)

    normalized_score_dict = {k: round(v,3) for k, v in sorted(normalized_score_dict.items(), key=lambda item: item[1])}
    # print(normalized_score_dict)
    return normalized_score_dict

In [ ]:
def calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score):
  score_dict = {
              'anticipation':0,
              'anger':0,
              'fear':0,
              'sadness':0,
              'trust':0,
              'joy':0,
              'surprise':0,
              'disgust':0
              }

  #Scoring Mechanism
  for i in range(0,len(neaarest_neighbs_words)):
    distance = 0
    if neaarest_neighbs_distance[i] == 0 :
      distance = 1
    else :
      distance = (1/neaarest_neighbs_distance[i])
    
    score = (distance*(neaarest_neighbs_intensity_score[i]))  

    score_dict[neaarest_neighbs_labels[i]]=score_dict[neaarest_neighbs_labels[i]]+score


  #Normalising Mechanism
  normalized_score_dict = score_dict.copy()
  # for k in score_dict.keys():
  #   if score_dict[k] ==0:
  #     continue
  #     # del normalized_score_dict[k]
  #   else:
  #     normalized_score_dict[k]
  #     # normalized_score_dict[k] = round((score_dict[k]/score_max),3)

  return normalized_score_dict

In [ ]:
import operator

def dict_to_result(emotion_dict):
  if (all(value == 0 for value in emotion_dict.values())):
    highest_8 = None
  else:
    highest_8 = max(emotion_dict.items(), key=operator.itemgetter(1))[0]
  return highest_8

# result = emotion_candidates_recognition('That was good',1)
# x = dict_to_result(result)

In [ ]:
dff.tail()

,text,embedding,label,unit_id,index
784,turkish bombardment kills civilians in syria,"[-0.4413837492465973, 0.17336057126522064, -1....",0,1965511221,784
785,martyrs as terrorists shell aleppos salah eddin,"[-0.18589265644550323, 0.01940799504518509, -1...",0,1965511222,785
786,chemical attack kills five syrians in aleppo sana,"[-0.20558828115463257, -0.15264035761356354, -...",1,1965511224,786
787,killed as russian military chopper shot down i...,"[-0.47541752457618713, -0.25362876057624817, -...",0,1965511226,787
788,syrian army kills isil terrorists in deir ezzor,"[-0.43960684537887573, -0.058433108031749725, ...",0,1965511231,788


In [ ]:
dff['emotion_score_dict'] = dff['index'].parallel_apply(get_nearest_neighbours)

In [ ]:
dff['highest_eight_label'] = dff['emotion_score_dict'].parallel_apply(dict_to_result)

In [ ]:
eight_label = dff['emotion_score_dict'].to_list()

dff["anger"]=[i["anger"] for i in eight_label]
dff["anticipation"]=[i["anticipation"] for i in eight_label]
dff["disgust"]=[i["disgust"] for i in eight_label]
dff["fear"]=[i["fear"] for i in eight_label]
dff["joy"]=[i["joy"] for i in eight_label]
dff["sadness"]=[i["sadness"] for i in eight_label]
dff["surprise"]=[i["surprise"] for i in eight_label]
dff["trust"]=[i["trust"] for i in eight_label]

In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 3118 non-null   object 
 1   embedding            3118 non-null   object 
 2   label                3118 non-null   int64  
 3   id                   3118 non-null   int64  
 4   subcategory          3118 non-null   object 
 5   index                3118 non-null   int64  
 6   emotion_score_dict   3118 non-null   object 
 7   highest_eight_label  3118 non-null   object 
 8   anger                3118 non-null   float64
 9   anticipation         3118 non-null   float64
 10  disgust              3118 non-null   float64
 11  fear                 3118 non-null   float64
 12  joy                  3118 non-null   float64
 13  sadness              3118 non-null   float64
 14  surprise             3118 non-null   float64
 15  trust                3118 non-null   f

In [ ]:
dff.head()

,text,embedding,label,id,subcategory,index,emotion_score_dict,highest_eight_label,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,facebook deletes michigan anti lockdown group ...,"[-0.5331893563270569, -0.0757744312286377, -0....",1,0,false news,0,"{'anticipation': 0, 'joy': 0, 'trust': 0.702, ...",anger,19.038,0.000,3.090,12.925,0.000,6.788,2.891,0.702
1,other viewpoints covid is worse than the flu,"[-0.7762630581855774, -0.12266916781663895, -0...",0,1,true,1,"{'anticipation': 0, 'surprise': 1.025, 'trust'...",anger,10.455,0.000,10.186,1.954,2.185,7.866,1.025,1.142
2,bermuda s covid cases surpass,"[-0.0547846220433712, -0.5317873954772949, 0.4...",0,2,true,2,"{'disgust': 1.677, 'sadness': 1.935, 'anticipa...",trust,5.328,2.234,1.677,5.131,4.871,1.935,2.686,12.183
3,purdue university says students face close to ...,"[-0.7069640755653381, -0.3513520658016205, -0....",1,3,partially false,3,"{'surprise': 0, 'disgust': 0.392, 'sadness': 0...",trust,4.550,10.722,0.392,5.362,1.625,0.738,0.000,12.295
4,the high cost of locking down america weve see...,"[-1.2972077131271362, 0.0439135767519474, -0.4...",1,4,false news,4,"{'joy': 0, 'trust': 0.579, 'anticipation': 1.3...",sadness,3.818,1.322,5.431,7.673,0.000,10.953,2.982,0.579


In [ ]:
out_dff = dff.copy()
# out_dff.drop(['embedding'], axis=1, inplace=True)

In [ ]:
out_dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 3118 non-null   object 
 1   embedding            3118 non-null   object 
 2   label                3118 non-null   int64  
 3   id                   3118 non-null   int64  
 4   subcategory          3118 non-null   object 
 5   index                3118 non-null   int64  
 6   emotion_score_dict   3118 non-null   object 
 7   highest_eight_label  3118 non-null   object 
 8   anger                3118 non-null   float64
 9   anticipation         3118 non-null   float64
 10  disgust              3118 non-null   float64
 11  fear                 3118 non-null   float64
 12  joy                  3118 non-null   float64
 13  sadness              3118 non-null   float64
 14  surprise             3118 non-null   float64
 15  trust                3118 non-null   f

In [ ]:
out_dff.head()

,text,embedding,label,id,subcategory,emotion_score_dict,highest_eight_label,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,facebook deletes michigan anti lockdown group ...,"[-0.5331893563270569, -0.0757744312286377, -0....",1,0,false news,"{'anticipation': 0, 'joy': 0, 'trust': 0.702, ...",anger,19.038,0.000,3.090,12.925,0.000,6.788,2.891,0.702
1,other viewpoints covid is worse than the flu,"[-0.7762630581855774, -0.12266916781663895, -0...",0,1,true,"{'anticipation': 0, 'surprise': 1.025, 'trust'...",anger,10.455,0.000,10.186,1.954,2.185,7.866,1.025,1.142
2,bermuda s covid cases surpass,"[-0.0547846220433712, -0.5317873954772949, 0.4...",0,2,true,"{'disgust': 1.677, 'sadness': 1.935, 'anticipa...",trust,5.328,2.234,1.677,5.131,4.871,1.935,2.686,12.183
3,purdue university says students face close to ...,"[-0.7069640755653381, -0.3513520658016205, -0....",1,3,partially false,"{'surprise': 0, 'disgust': 0.392, 'sadness': 0...",trust,4.550,10.722,0.392,5.362,1.625,0.738,0.000,12.295
4,the high cost of locking down america weve see...,"[-1.2972077131271362, 0.0439135767519474, -0.4...",1,4,false news,"{'joy': 0, 'trust': 0.579, 'anticipation': 1.3...",sadness,3.818,1.322,5.431,7.673,0.000,10.953,2.982,0.579


In [ ]:
out_dff.drop(['index'], axis = 1, inplace = True)

In [ ]:
out_dff.to_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/COVID19_test_title_emotion_scores_modified.csv",index=False)

In [ ]:
dfff = pd.read_csv('/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/COVID19_test/COVID19_test_title_emotion_scores_modified.csv')

dfff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 3118 non-null   object 
 1   embedding            3118 non-null   object 
 2   label                3118 non-null   int64  
 3   id                   3118 non-null   int64  
 4   subcategory          3118 non-null   object 
 5   emotion_score_dict   3118 non-null   object 
 6   highest_eight_label  3118 non-null   object 
 7   anger                3118 non-null   float64
 8   anticipation         3118 non-null   float64
 9   disgust              3118 non-null   float64
 10  fear                 3118 non-null   float64
 11  joy                  3118 non-null   float64
 12  sadness              3118 non-null   float64
 13  surprise             3118 non-null   float64
 14  trust                3118 non-null   float64
dtypes: float64(8), int64(2), object(5)
mem

In [ ]:
dfff.head()

,text,embedding,label,id,subcategory,emotion_score_dict,highest_eight_label,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,facebook deletes michigan anti lockdown group ...,"[-0.5331893563270569, -0.0757744312286377, -0....",1,0,false news,"{'anticipation': 0, 'joy': 0, 'trust': 0.702, ...",anger,19.038,0.000,3.090,12.925,0.000,6.788,2.891,0.702
1,other viewpoints covid is worse than the flu,"[-0.7762630581855774, -0.12266916781663895, -0...",0,1,true,"{'anticipation': 0, 'surprise': 1.025, 'trust'...",anger,10.455,0.000,10.186,1.954,2.185,7.866,1.025,1.142
2,bermuda s covid cases surpass,"[-0.0547846220433712, -0.5317873954772949, 0.4...",0,2,true,"{'disgust': 1.677, 'sadness': 1.935, 'anticipa...",trust,5.328,2.234,1.677,5.131,4.871,1.935,2.686,12.183
3,purdue university says students face close to ...,"[-0.7069640755653381, -0.3513520658016205, -0....",1,3,partially false,"{'surprise': 0, 'disgust': 0.392, 'sadness': 0...",trust,4.550,10.722,0.392,5.362,1.625,0.738,0.000,12.295
4,the high cost of locking down america weve see...,"[-1.2972077131271362, 0.0439135767519474, -0.4...",1,4,false news,"{'joy': 0, 'trust': 0.579, 'anticipation': 1.3...",sadness,3.818,1.322,5.431,7.673,0.000,10.953,2.982,0.579
